# PyTorch - homework 1


Please run the whole notebook with your code and submit the `.ipynb` file that includes your answers. 

In [1]:
from termcolor import colored

student_name="Ivan Christian"
student_number="1003056"

print(colored("Homework by "  + student_name + ', number: ' + student_number,'red'))

Homework by Ivan Christian, number: 1003056


 ## Question 1 -- matrix multiplication

Implement the following mathematical operation on both the CPU and GPU (use Google Colab or another cloud service if you don't have a GPU in your computer). Print:

a) which type of GPU card you have and 

b) show the computation time for both CPU and GPU (using PyTorch). 

c) How much % fast is the GPU? 

 The operation to implement is the dot product $C = B * A^T$

 whereby $A$ is a random matrix of size $30,000 \times 1000$ and $B$ is a random matrix of size $3000 \times 1000$. In addition to the required information asked above:
 
 d) please also print the resulting two $C$ matrices (they should be the same btw). 
 



In [2]:
# implement solution here
import torch

device_name = torch.cuda.get_device_name(0)

print(f'Type of GPU card I have : {device_name}')

## Dot product C = B* A.transpose

A = torch.rand((30000, 1000))

B = torch.rand((3000, 1000))

start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)
## GPU 

device = 'cuda'
start.record()
C = torch.matmul(B.to(device), torch.transpose(A, 0, 1).to(device))
end.record()
torch.cuda.synchronize()
gpu_time_elapsed = start.elapsed_time(end)

print(f'{device} : {C}, shape : {C.shape}, time_elapsed : {gpu_time_elapsed}')

## CPU

device = 'cpu'
start.record()
C = torch.matmul(B.to(device), torch.transpose(A, 0, 1).to(device))
end.record()
torch.cuda.synchronize()
cpu_time_elapsed = start.elapsed_time(end)
print(f'{device} : {C}, shape : {C.shape}, time_elapsed : {cpu_time_elapsed}')

diff = (cpu_time_elapsed - gpu_time_elapsed)/ gpu_time_elapsed

print(f'GPU is {diff*100}% faster')

Type of GPU card I have : GeForce RTX 2060
cuda : tensor([[257.9044, 242.9455, 247.7620,  ..., 238.5240, 250.5987, 254.3756],
        [255.2494, 242.4609, 245.0562,  ..., 241.0914, 250.8923, 244.3374],
        [262.7208, 253.6382, 255.5354,  ..., 251.8262, 261.0026, 265.3585],
        ...,
        [259.0933, 252.1954, 252.6086,  ..., 246.3110, 255.3872, 255.7017],
        [250.2514, 246.3637, 247.5396,  ..., 239.7097, 250.0018, 249.2021],
        [263.7563, 245.3566, 255.5344,  ..., 242.8696, 260.2724, 260.4454]],
       device='cuda:0'), shape : torch.Size([3000, 30000]), time_elapsed : 431.167724609375
cpu : tensor([[257.9044, 242.9454, 247.7620,  ..., 238.5241, 250.5985, 254.3757],
        [255.2494, 242.4608, 245.0563,  ..., 241.0913, 250.8923, 244.3375],
        [262.7208, 253.6382, 255.5353,  ..., 251.8262, 261.0026, 265.3584],
        ...,
        [259.0934, 252.1954, 252.6087,  ..., 246.3110, 255.3870, 255.7018],
        [250.2516, 246.3635, 247.5394,  ..., 239.7097, 250.0017, 

## Question 2 - grad


Find the gradient (partial derivatives) of the function $g(w)$ below. 

Let  $w=[w_1,w_2]^T$

Consider  $g(w)=2w_1w_2+w_2cos(w_1)$

a) In PyTorch, compute:   $\Delta_w g(w)$ 

 and verify that $\Delta_w g([\pi,1])=[2,2*\pi−1]^T$ using the grad function, whereby the first position is the partial for $w_1$ and the second position is the partial for $w_2$. 

b) You can also write a function to manually calculate these partial derivatives! You can review your differential equations math at [here](https://www.wolframalpha.com/input/?i=derivative+y+cos%28x%29) and implement this is a second function below to verify that it comes to the same solution. 


In [3]:
# write your solution here

import numpy as np

pi = np.pi

w = torch.tensor([pi,1.],requires_grad=True).reshape(-1,1)
g_w = 2 * w[0] * w[1]  + w[1] * torch.cos(w[0])

dw_auto = torch.autograd.grad(g_w, w)
print(f'Autograd Partial Differentiation : {dw_auto}')

# autograd

# manual partial diff
dw1 = 2 * w[1] - torch.sin(w[0])*w[1]
dw2 = 2 * w[0] + torch.cos(w[0])

dg_manual = torch.Tensor([dw1,dw2])
print(f'Manual partial differntiation : {dg_manual}')

Autograd Partial Differentiation : (tensor([[2.0000],
        [5.2832]]),)
Manual partial differntiation : tensor([2.0000, 5.2832])


## Question 3 - dance hit song prediction

Implement logistic regression in PyTorch for the following dance hit song prediction training dataset: 
https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030training.csv

 * Input variables: a number of audio features (most already standardized so don't worry about that)
 * Target variable: Topclass1030: 
   * 1 means it was a top 10 hit song; 
   * 0 means it never went above top 30 position.

This dataset is derived from my paper on dance hit song prediction, for full description of features have a look at https://arxiv.org/abs/1905.08076. 

Print the evolution of the loss every few epochs and train the model until it converges. 
 
 After training the logistic regression model, calculate the prediction accuracy on the test set: 
 https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030test.csv








In [4]:
# Your code here

!pip install wget
import wget
import os

import pandas as pd 

train_url = 'https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030training.csv'
test_url = 'https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030test.csv'


if not os.path.exists('herremans_hit_1030training.csv') and not os.path.exists('herremans_hit_1030test.csv'):
    train_data = wget.download(train_url) 
    test_data = wget.download(test_url)
else:
    train_data = 'herremans_hit_1030training.csv'
    test_data = 'herremans_hit_1030test.csv'

# load data

train_data = pd.read_csv(train_data)
test_data = pd.read_csv(test_data)

# define logistic regression model

import torch.nn as nn
import torch.nn.functional as F

class LogisticRegression(nn.Module):
  # input_size: Dimensionality of input feature vector.
  # num_classes: The number of classes in the classification problem.
    def __init__(self, input_size, num_classes):
    # Always call the superclass (nn.Module) constructor first!
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
    def forward(self, x):
        out = self.linear(x)
        out = torch.sigmoid(out)
        return out

# train model

device = 'cuda'
epochs = 5001

num_outputs = 1
num_input_features = 49

lr_rate = 0.001
loss_function = nn.BCELoss()
logreg_clf = LogisticRegression(num_input_features, num_outputs).to(device)
optimizer = torch.optim.SGD(logreg_clf.parameters(), lr=lr_rate)

for epoch in range(epochs):
    x_data = torch.FloatTensor(train_data.loc[:, train_data.columns != 'Topclass1030'].values).to(device)
    y_true = torch.FloatTensor(train_data['Topclass1030']).to(device)
    
    optimizer.zero_grad()
    
    y_pred = logreg_clf(x_data)
    
    loss = loss_function(y_pred, y_true).to(device) #calculate the loss
    loss.backward() #backprop
    optimizer.step()
    if epoch % 500 == 0:
        print ("Epoch: {0}, Loss: {1}, ".format(epoch,loss))

C:\Users\chris\Anaconda3\envs\tf_gpu\lib\site-packages\torch\nn\modules\loss.py:498: UserWarning: Using a target size (torch.Size([321])) that is different to the input size (torch.Size([321, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


Epoch: 0, Loss: 0.7169710993766785, 
Epoch: 500, Loss: 0.6137398481369019, 
Epoch: 1000, Loss: 0.5900614261627197, 
Epoch: 1500, Loss: 0.5772870182991028, 
Epoch: 2000, Loss: 0.5682091116905212, 
Epoch: 2500, Loss: 0.5611566305160522, 
Epoch: 3000, Loss: 0.5554267764091492, 
Epoch: 3500, Loss: 0.5506284832954407, 
Epoch: 4000, Loss: 0.5465198159217834, 
Epoch: 4500, Loss: 0.542942464351654, 
Epoch: 5000, Loss: 0.5397874712944031, 


Run the below code to test the accuracy of your model on the training set: 

In [8]:
import pandas as pd 

device = 'cuda'

test = pd.read_csv('herremans_hit_1030test.csv')
labels = test.iloc[:,-1]
test = test.drop('Topclass1030', axis=1)
testdata = torch.Tensor(test.values).to(device)
testlabels = torch.Tensor(labels.values).view(-1,1).to(device)

TP = 0
TN = 0
FN = 0
FP = 0



for i in range(0, testdata.size()[0]): 
    Xtest = testdata[i]
    y_hat = logreg_clf(Xtest)
    if y_hat > 0.5:
        prediction = 1
    else: 
        prediction = 0

    if (prediction == testlabels[i]):
        if (prediction == 1):
            TP += 1
        else: 
            TN += 1

    else:
        if (prediction == 1):
            FP += 1
        else: 
            FN += 1

print("True Positives: {0}, True Negatives: {1}".format(TP, TN))
print("False Positives: {0}, False Negatives: {1}".format(FP, FN))
rate = TP/(FN+TP)
print("Class specific accuracy of correctly predicting a hit song is {0}".format(rate))

True Positives: 44, True Negatives: 13
False Positives: 16, False Negatives: 6
Class specific accuracy of correctly predicting a hit song is 0.88
